___
# COMP1804 Lab 2 - Data Pre-processing



**Learning Objectives:**
 *  Understand the stages of data pre-processing.
 *  Use Python for pre-processing. 
 *  Practice the different stages of data pre-processing in Python.
___


Credits:
Tutorial adapted from last year's one, delivered by Dimitris Kollias

## 1. Loading the Dataset


For this Lab, I have used a subset of the Loan Prediction dataset. You can download the training and testing dataset from Moodle: [Download Data](https://moodlecurrent.gre.ac.uk/mod/resource/view.php?id=1730057)

Note : Testing data that you are provided is the subset of the training data from Loan Prediction problem.

 

Now, lets get started by importing important packages and the dataset.

**1.1 Import the necessary Python modules**

In [ ]:
# Load python modules

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import model_selection


In [ ]:
from IPython.display import HTML
def pretty_print_df(value_counts_):
  "Quick function to display value counts more nicely"
  display(HTML(pd.DataFrame(value_counts_).to_html()))


**1.2 Load Dataset **

Note: Download the csv files from the URL to your local drive and load from there as shown in the code below.

We are using pandas to load the data. We will also use pandas next to explore the data both with descriptive statistics and data visualization.


In [ ]:
# Load dataset from local drive (for colab notebook)
from google.colab import files
import io

uploaded = files.upload()    # Will prompt you to select file: remember to choose the right one!
train_dataset = pd.read_csv(io.BytesIO(uploaded['X_train.csv']))

uploaded = files.upload()    # Will prompt you to select file
train_labels = pd.read_csv(io.BytesIO(uploaded['Y_train.csv']))

uploaded = files.upload()    # Will prompt you to select file
test_dataset = pd.read_csv(io.BytesIO(uploaded['X_test.csv']))

uploaded = files.upload()    # Will prompt you to select file
test_labels = pd.read_csv(io.BytesIO(uploaded['Y_test.csv']))




Saving X_train.csv to X_train.csv


Saving Y_train.csv to Y_train.csv


Saving X_test.csv to X_test.csv


Saving Y_test.csv to Y_test.csv


**1.2.1 Inspect Dataset **

**1.2.1.1 Dimensions of Dataset **


In [ ]:
# Training data
# shape of input: 
# The number or rows is the number of data points
# The number or columns is the number of features
print(train_dataset.shape)
# shape of output:
# The number or rows is the number of data points (should be the same as before!)
# The number or columns is the number of labels we want to predict
print(train_labels.shape)

(391, 12)
(391, 1)


In [ ]:
# Test data
# shape of input: 
# The number or rows is the number of data points
# The number or columns is the number of features (should be the same as for the training!)
print(test_dataset.shape)
# shape of output:
# The number or rows is the number of data points (should be the same as before!)
# The number or columns is the number of labels we want to predict
print(test_labels.shape)

(96, 12)
(96, 1)


In [ ]:
# list of column titles 
print(train_dataset.columns)
print(train_labels.columns)

Index(['Loan_ID', 'Gender', 'Married', 'Dependents', 'Education',
       'Self_Employed', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area'],
      dtype='object')
Index(['Target'], dtype='object')


In [ ]:
# list of column (field) data types
print(train_dataset.dtypes)
print(train_labels.dtypes)

# Note: object is 

Loan_ID               object
Gender                object
Married               object
Dependents            object
Education             object
Self_Employed         object
ApplicantIncome      float64
CoapplicantIncome    float64
LoanAmount           float64
Loan_Amount_Term     float64
Credit_History       float64
Property_Area         object
dtype: object
Target    object
dtype: object


**1.2.1.2 Take a peek at the Dataset **

Note that if you see "NaN", it means "Not a Number". It is not the same as 0. Python replaces empty/missing fields in the data with "NaN".


In [ ]:
# you can show the first N rows in a dataframe with the function "head"
train_dataset.head(10)
# note how the first row has a missing value! We'll get back to it.
# Also notice how the first two rows are almost identical, aside from the Loan_ID and the missing value
# Could this be the same application submitted twice because the first time someone forgot to add the income?

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,LP001031,Male,No,0,Graduate,No,NaN,0.0,125.0,360.0,1.0,Urban
1,LP001032,Male,No,0,Graduate,No,4950.0,0.0,125.0,360.0,1.0,Urban
2,LP001824,Male,Yes,1,Graduate,No,2882.0,1843.0,123.0,480.0,1.0,Semiurban
3,LP002928,Male,Yes,0,Graduate,No,3000.0,3416.0,56.0,180.0,1.0,Semiurban
4,LP001814,Male,Yes,2,Graduate,No,9703.0,0.0,112.0,360.0,1.0,Urban
5,LP002244,Male,Yes,0,Graduate,No,2333.0,2417.0,136.0,360.0,1.0,Urban
6,LP001854,Male,Yes,3+,Graduate,No,5250.0,0.0,94.0,360.0,1.0,Urban
7,LP002505,Male,Yes,0,Graduate,No,4333.0,2451.0,110.0,360.0,1.0,Urban
8,LP002862,other,Yes,2,Not Graduate,No,6125.0,1625.0,187.0,480.0,1.0,Semiurban
9,LP001630,Male,No,0,Not Graduate,No,2333.0,1451.0,102.0,480.0,0.0,Urban


In [ ]:
# you can also show a random subset of the data with the function "sample":
train_dataset.sample(10)

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
329,LP001900,Male,Yes,1,Graduate,No,2750.0,1842.0,115.0,360.0,1.0,Semiurban
303,LP002116,Female,No,0,Graduate,No,2378.0,0.0,46.0,360.0,1.0,Rural
143,LP002836,Male,No,0,Graduate,No,3333.0,0.0,70.0,360.0,1.0,Urban
95,LP001579,Male,No,0,Graduate,No,2237.0,0.0,63.0,480.0,0.0,Semiurban
76,LP002840,Female,No,0,Graduate,No,2378.0,0.0,9.0,360.0,1.0,Urban
126,LP002813,Female,Yes,1,Graduate,Yes,19484.0,0.0,600.0,360.0,1.0,Semiurban
177,LP002772,Male,No,0,Graduate,No,2526.0,1783.0,145.0,360.0,1.0,Rural
88,LP001807,Male,Yes,2,Graduate,Yes,6250.0,1300.0,108.0,360.0,1.0,Rural
101,LP002317,Male,Yes,3+,Graduate,No,81000.0,0.0,360.0,360.0,0.0,Rural
250,LP002211,Male,Yes,0,Graduate,No,4817.0,923.0,120.0,180.0,1.0,Urban


## 2. Data quality assessment and Exploratory Data Analysis

It is good practice to spend some time exploring the data to find out any issues as early as possible.

There can be many data quality issues, including invalid/inconsistent features or data points, unnecessary features, missing values, and more.

For large dataset it is impractical to go through each data point one by one (although it is not unheard of for small to medium datasets). So, we need to define some rules for checking. We can do this more easily if we know (or can guess) what to expect from the data, but this might not always be possible. Exploratory data analysis often goes together with data quality assessment. After all, we need to explore the data to spot any issues!

(note that we will explore missing data more in-depth later on in the tutorial).

First EDA steps: use Pandas `describe()` function to get some high-level statistics about the data

For NUMERICAL features, we can get the following info:

*   count = Count number of non-NA/null observations for a feature (see also below).
*   mean = average value of each feature.
*   std = standard deviation of the values in each feature
*   min = Minium of the values in the feature.
*   max = Maximum of the values in the feature.
*   25%, 50%, 75% = 25%, 50%, 75% percentiles in each feature. 50% percentile=median

We will get back to the percentile later on, but here is a good explanation: https://simple.wikipedia.org/wiki/Percentile.

In [ ]:
# Summary statistics for numerical features
train_dataset.describe()


,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History
count,387.000000,388.000000,390.000000,389.000000,390.000000
mean,5134.046512,1579.234330,140.589744,341.861183,0.856410
std,5288.031010,2622.889598,75.797701,65.961182,0.351124
min,150.000000,0.000000,-21.000000,36.000000,0.000000
25%,2888.500000,0.000000,100.000000,360.000000,1.000000
50%,3887.000000,1188.500000,126.500000,360.000000,1.000000
75%,5818.500000,2277.000000,163.500000,360.000000,1.000000
max,81000.000000,33837.000000,600.000000,480.000000,1.000000


For CATEGORICAL features, we want to use the Pandas function `value_counts()` to obtain information about how many categories there are and how many instances of each.

In [ ]:
# let's list all categorical features
categorical_columns= ['Gender', 'Married', 'Dependents', 'Education',
       'Self_Employed', 'Property_Area']
# Question: why have we not included Loan_ID?

# let's get the categories and their count for each feature
for col in categorical_columns:
  print(f"Categories and number of occurrences for '{col}'")
  pretty_print_df(train_dataset[col].value_counts())
  print()

Categories and number of occurrences for 'Gender'


,Gender
Male,316
Female,63
other,11



Categories and number of occurrences for 'Married'


,Married
Yes,254
No,136



Categories and number of occurrences for 'Dependents'


...

,Education
Graduate,312
Not Graduate,78



Categories and number of occurrences for 'Self_Employed'


,Self_Employed
No,333
Yes,54
no,4



Categories and number of occurrences for 'Property_Area'


,Property_Area
Semiurban,149
Urban,125
Rural,113
London,3


Wait! Some things don't make much sense:


1.   Why is there a category "London" in "Property_Area"? This feature describes the type of area someone lives in, not the city. 
2.   Why is the minimum "Loan Amount" a negative value?
3.   Are "no" and "No" the same category for the "Self_Employed" feature. We can guess that yes, they do mean the same thing.


These are all mistakes that may have arisen from human or machine errors. How do we fix them?

Well, negative income and a wrong value for a given category really means that we do not have information about that specific value. In practice, it's as if it were a missing value, we might as well replace it with on. The mismatched categories can be merged, instead.

Two notes:
1. In some cases, the wrong values still give us relevant information. For example, if we could be sure that "London" is actually the city where the applicant lives, we could reasonably assume that the corresponding "Property_Area" value would be "Urban". However, these decisions are highly case-specific.
2. The "Other" in the gender category is not a mistake. Many data collection processes do include options for genders that are different from "Male" and "Female".

In [ ]:
# replace 'yes' with 'Yes' for 'self_employed'
train_dataset.loc[train_dataset.Self_Employed=='no','Self_Employed'] = 'No'

# replace negative loan amount values with NaN 
# (we use np.nan because that's how the other missing values are represented in this dataset)
train_dataset.loc[train_dataset.LoanAmount<0,'LoanAmount'] = np.nan

# replace out of scope Property_Area values with NaN
accepted_property_areas = ['Urban','Rural','Semiurban']
train_dataset.loc[train_dataset.Property_Area.map(lambda x: x not in accepted_property_areas),'Property_Area'] = np.nan


## 3. Managing Missing Data

Sometimes you may find some data are missing in the dataset. If the missing values are not handled properly inaccurate inference about the data may result. Due to improper handling, the result obtained will differ from ones where the missing values are present. 

Since missing values can tangibly reduce prediction accuracy, this step needs to be a priority. In terms of machine learning, assumed or approximated values are “more appropriate” for an algorithm than just missing ones.  Even if you don’t know the exact value, methods exist to better “assume” which value is missing or bypass the issue. However, keep in mind that this might also be introducing biases in your dataset, especially when data is not missing at random (For example, some people might be more reluctant than other to disclose their, let's say, immigration status, for multiple reasons. In this case, an "assumed" value might not work as well).

So how to сlean the data here? Choosing the right approach also heavily depends on data and the domain you have:
* Substitute missing values with dummy values, e.g. n/a for categorical or 0 for numerical values. (This will retain the information that a data value is missing, which might be important in some cases.)
* Substitute the missing numerical values with mean figures.
* For categorical values, you can also use the most frequent items to fill in.

First, let's find out how many missing values (or NaN values) there are in each feature, using Pandas `isna()` function. 

In [ ]:
# Number of missing values per column
train_dataset.isna().sum()

Loan_ID              0
Gender               1
Married              1
Dependents           1
Education            1
Self_Employed        0
ApplicantIncome      4
CoapplicantIncome    3
LoanAmount           3
Loan_Amount_Term     2
Credit_History       1
Property_Area        4
dtype: int64

### 3.1 Removing Missing Data

So how can we handle missing data. One obvious idea is to remove the lines (observations recorded in the rows) where there is some missing data. That is ok if you’ve got large datasets. The dataset we have here is considerably small, so removing data will delete relevant information and can have a crucial impact. We could therefore need to look for alternative methods to deal with the missing data.

The decision to remove data will depend on the size of dataset and the problem domain (type of data collected).

Assuming it was appropriate to remove observation rows, the following code will help. It uses pandas function `dropna()`.



**2.1.1 Remove all rows that contain missing data**


In [ ]:
# remove all rows with missing data
# dropna removes all rows that contain at least one missing value
print(f'Original dataset length: {len(train_dataset)}')
reduced_train_dataset = train_dataset.dropna()
print(f'Dataset length after removing missing rows: {len(reduced_train_dataset)}')
print()
print(reduced_train_dataset[['Loan_ID']].head(5))
train_dataset.head(5)
# Note how the rows order has changed because we dropped the first one!


Original dataset length: 391
Dataset length after removing missing rows: 379

    Loan_ID
1  LP001032
2  LP001824
3  LP002928
4  LP001814
5  LP002244


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,LP001031,Male,No,0,Graduate,No,NaN,0.0,125.0,360.0,1.0,Urban
1,LP001032,Male,No,0,Graduate,No,4950.0,0.0,125.0,360.0,1.0,Urban
2,LP001824,Male,Yes,1,Graduate,No,2882.0,1843.0,123.0,480.0,1.0,Semiurban
3,LP002928,Male,Yes,0,Graduate,No,3000.0,3416.0,56.0,180.0,1.0,Semiurban
4,LP001814,Male,Yes,2,Graduate,No,9703.0,0.0,112.0,360.0,1.0,Urban


**2.1.2 Remove specific rows**

We can drop specific rows by passing index labels to the drop method.

In [ ]:
# remove selected rows (e.g. the third row)

print(train_dataset.drop([2]).head(5))
train_dataset.head(5)


    Loan_ID Gender Married  ... Loan_Amount_Term Credit_History Property_Area
0  LP001031   Male      No  ...            360.0            1.0         Urban
1  LP001032   Male      No  ...            360.0            1.0         Urban
3  LP002928   Male     Yes  ...            180.0            1.0     Semiurban
4  LP001814   Male     Yes  ...            360.0            1.0         Urban
5  LP002244   Male     Yes  ...            360.0            1.0         Urban

[5 rows x 12 columns]


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,LP001031,Male,No,0,Graduate,No,NaN,0.0,125.0,360.0,1.0,Urban
1,LP001032,Male,No,0,Graduate,No,4950.0,0.0,125.0,360.0,1.0,Urban
2,LP001824,Male,Yes,1,Graduate,No,2882.0,1843.0,123.0,480.0,1.0,Semiurban
3,LP002928,Male,Yes,0,Graduate,No,3000.0,3416.0,56.0,180.0,1.0,Semiurban
4,LP001814,Male,Yes,2,Graduate,No,9703.0,0.0,112.0,360.0,1.0,Urban


**2.1.3 Remove specific columns**

Usually you would drop particular columns especially if all or most of its values are missing. The drop method can also be used here with parameters to define the column title and axis=1 to denote that we want to drop a column.


In [ ]:
# remove selected column

print(train_dataset.drop("Gender", axis=1))

      Loan_ID Married  ... Credit_History Property_Area
0    LP001031      No  ...            1.0         Urban
1    LP001032      No  ...            1.0         Urban
2    LP001824     Yes  ...            1.0     Semiurban
3    LP002928     Yes  ...            1.0     Semiurban
4    LP001814     Yes  ...            1.0         Urban
..        ...     ...  ...            ...           ...
386  LP001841      No  ...            1.0         Rural
387  LP002820     Yes  ...            1.0         Rural
388  LP001744      No  ...            1.0     Semiurban
389  LP001552     Yes  ...            1.0     Semiurban
390  LP001553     NaN  ...            NaN     Semiurban

[391 rows x 11 columns]


In [ ]:
# remove multiple selected columns

print(train_dataset.drop(["Gender", "Married"], axis=1))


      Loan_ID Dependents  ... Credit_History Property_Area
0    LP001031          0  ...            1.0         Urban
1    LP001032          0  ...            1.0         Urban
2    LP001824          1  ...            1.0     Semiurban
3    LP002928          0  ...            1.0     Semiurban
4    LP001814          2  ...            1.0         Urban
..        ...        ...  ...            ...           ...
386  LP001841          0  ...            1.0         Rural
387  LP002820          0  ...            1.0         Rural
388  LP001744          0  ...            1.0     Semiurban
389  LP001552          0  ...            1.0     Semiurban
390  LP001553          0  ...            NaN     Semiurban

[391 rows x 10 columns]


** 2.1.4 Remove all rows where data satisfies a condition in a particular column**

Drop all rows where Married is "No" (or a NaN value). Since there are two categories in the Married column, that is equivalent to filtering the dataframe where Married = Yes.


In [ ]:
# remove all rows where Married is No (or NaN).

print(train_dataset[train_dataset["Married"] == "Yes"])

      Loan_ID Gender Married  ... Loan_Amount_Term Credit_History Property_Area
2    LP001824   Male     Yes  ...            480.0            1.0     Semiurban
3    LP002928   Male     Yes  ...            180.0            1.0     Semiurban
4    LP001814   Male     Yes  ...            360.0            1.0         Urban
5    LP002244   Male     Yes  ...            360.0            1.0         Urban
6    LP001854   Male     Yes  ...            360.0            1.0         Urban
..        ...    ...     ...  ...              ...            ...           ...
383  LP001401   Male     Yes  ...            180.0            1.0         Rural
384  LP002484   Male     Yes  ...            180.0            1.0         Urban
385  LP002585   Male     Yes  ...            360.0            0.0         Rural
387  LP002820   Male     Yes  ...            360.0            1.0         Rural
389  LP001552   Male     Yes  ...            360.0            1.0     Semiurban

[254 rows x 12 columns]


**2.1.5 Remove all rows where data is missing in a particular column**

Remove all rows where a particular column has a missing value. This will result in a dataset that has no missing values in that particular column. It may still contain missing values in other columns.


In [ ]:
# remove all rows where ApplicantIncome is missing.
# 1. check to see if there are missing data in the ApplicantIncome column
print('Missing ApplicantIncome data')
print(train_dataset["ApplicantIncome"].isnull().sum())

print()

# 2. drop all rows where a value for ApplicantIncome is missing (isnull() is the same as isna())
print(train_dataset[train_dataset["ApplicantIncome"].notnull()])
# the first row contains a null and is removed

# alternative way: we can specify the column of interest as a parameter to the dropna function
train_dataset.dropna(subset=["ApplicantIncome"])


Missing ApplicantIncome data
4

      Loan_ID Gender Married  ... Loan_Amount_Term Credit_History Property_Area
1    LP001032   Male      No  ...            360.0            1.0         Urban
2    LP001824   Male     Yes  ...            480.0            1.0     Semiurban
3    LP002928   Male     Yes  ...            180.0            1.0     Semiurban
4    LP001814   Male     Yes  ...            360.0            1.0         Urban
5    LP002244   Male     Yes  ...            360.0            1.0         Urban
..        ...    ...     ...  ...              ...            ...           ...
386  LP001841   Male      No  ...            360.0            1.0         Rural
387  LP002820   Male     Yes  ...            360.0            1.0         Rural
388  LP001744   Male      No  ...            360.0            1.0     Semiurban
389  LP001552   Male     Yes  ...            360.0            1.0     Semiurban
390  LP001553   Male     NaN  ...            360.0            NaN     Semiurban

[387 ro

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
1,LP001032,Male,No,0,Graduate,No,4950.0,0.0,125.0,360.0,1.0,Urban
2,LP001824,Male,Yes,1,Graduate,No,2882.0,1843.0,123.0,480.0,1.0,Semiurban
3,LP002928,Male,Yes,0,Graduate,No,3000.0,3416.0,56.0,180.0,1.0,Semiurban
4,LP001814,Male,Yes,2,Graduate,No,9703.0,0.0,112.0,360.0,1.0,Urban
5,LP002244,Male,Yes,0,Graduate,No,2333.0,2417.0,136.0,360.0,1.0,Urban
...,...,...,...,...,...,...,...,...,...,...,...,...
386,LP001841,Male,No,0,Not Graduate,Yes,2583.0,2167.0,104.0,360.0,1.0,Rural
387,LP002820,Male,Yes,0,Graduate,No,5923.0,2054.0,211.0,360.0,1.0,Rural
388,LP001744,Male,No,0,Graduate,No,2971.0,2791.0,144.0,360.0,1.0,Semiurban
389,LP001552,Male,Yes,0,Graduate,No,4583.0,5625.0,255.0,360.0,1.0,Semiurban


### 3.2 Filling in Missing Data

As alternative to removing rows with missing data, you can estimate plausible values for the missing data instead. For example, replace the missing data within a column with a value equivalent to the mean of all the values in that column. That can also be applied to every feature where there is missing data.


We will use the scikit-learn library. The impute class allows us to manipulate the missing data.


At first we will create an object of the imputer class. 
The class has an argument called 'strategy' which shows what method to use to generate a value replacement for the missing value. 'mean' is the default value for strategy.

If “mean”, then replace missing values using the mean along each column. Can only be used with numeric data.

If “median”, then replace missing values using the median along each column. Can only be used with numeric data.

If “most_frequent”, then replace missing using the most frequent value along each column. Can be used with strings or numeric data. If there is more than one such value, only the smallest is returned.

If “constant”, then replace missing values with fill_value. Can be used with strings or numeric data.

*********************
Sklearn note.
There is a typical workflow with sklearn functions. These functions are typically objects which follow a specific pipeline:
1. create an instance of a given object (like the imputer class)
2. fit this instance on a training dataset
3. use the instance on all the relevant datasets (training, validation, testing, as applicable).

Remember this pipeline - it will appear again and again.
**********************

In [ ]:
# number of missing values for Gender and ApplicantIncome
print(train_dataset["Gender"].isnull().sum())

print(train_dataset["ApplicantIncome"].isnull().sum())



1
4


In [ ]:
# handling missing data
from sklearn.impute import SimpleImputer 

train_dataset_no_nans =  train_dataset.copy()

# 1. Imputer
imptr_num = SimpleImputer(missing_values = np.nan, strategy = 'mean')  


# 2. Fit the imputer object to the feature matrix (only for numeric features)
numerical_columns = ['ApplicantIncome', 'CoapplicantIncome',
                'LoanAmount', 'Loan_Amount_Term', 'Credit_History']
imptr_num = imptr_num.fit(train_dataset_no_nans[numerical_columns])

# 3. Call Transform to replace missing data in train_dataset (on specific columns) by the mean of the column to which that missing data belongs to
train_dataset_no_nans[numerical_columns] = \
  imptr_num.transform(train_dataset_no_nans[numerical_columns]) 

# note column ApplicantIncome in the first row --> before it was a missing value!
train_dataset_no_nans



,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,LP001031,Male,No,0,Graduate,No,5134.046512,0.0,125.0,360.0,1.00000,Urban
1,LP001032,Male,No,0,Graduate,No,4950.000000,0.0,125.0,360.0,1.00000,Urban
2,LP001824,Male,Yes,1,Graduate,No,2882.000000,1843.0,123.0,480.0,1.00000,Semiurban
3,LP002928,Male,Yes,0,Graduate,No,3000.000000,3416.0,56.0,180.0,1.00000,Semiurban
4,LP001814,Male,Yes,2,Graduate,No,9703.000000,0.0,112.0,360.0,1.00000,Urban
...,...,...,...,...,...,...,...,...,...,...,...,...
386,LP001841,Male,No,0,Not Graduate,Yes,2583.000000,2167.0,104.0,360.0,1.00000,Rural
387,LP002820,Male,Yes,0,Graduate,No,5923.000000,2054.0,211.0,360.0,1.00000,Rural
388,LP001744,Male,No,0,Graduate,No,2971.000000,2791.0,144.0,360.0,1.00000,Semiurban
389,LP001552,Male,Yes,0,Graduate,No,4583.000000,5625.0,255.0,360.0,1.00000,Semiurban


Now for an example on the categorical feature. This time we will use the "most_frequent" strategy.
Most people are not self employed, but is it safe to assume that we can replace all missing values with "not self-employed"?

In [ ]:
# 1. Imputer
imptr_empl = SimpleImputer(missing_values = np.nan, strategy = 'most_frequent')  

# 2. Fit the imputer object to the feature matrix
imptr_empl = imptr_empl.fit(train_dataset[['Self_Employed']])

# 3. Call Transform to replace missing data in train_dataset (on specific columns) by the mean of the column to which that missing data belongs to
train_dataset_no_nans[['Self_Employed']] = imptr_empl.transform(train_dataset[['Self_Employed']]) 



##############################################################################
##############################################################################
##############################################################################
##############################################################################




##  Exercise no 1

Fill in the missing values for all the categorical features together. 
Remember we made a list of categorical features above.

Call the SimpleImputer `imptr_cat`.

work on the: `train_dataset_no_nans`









In [ ]:
### insert code here; work on the: train_dataset_no_nans
# 1. Imputer
imptr_cat = SimpleImputer(missing_values = np.nan, strategy = 'most_frequent')  

# 2. Fit the imputer object to the feature matrix (only for categorical features)
imptr_cat = imptr_cat.fit(train_dataset_no_nans[categorical_columns])

# 3. Call Transform to replace missing data in train_dataset (on specific columns) by the mean of the column to which that missing data belongs to
train_dataset_no_nans[categorical_columns] = imptr_cat.transform(train_dataset_no_nans[categorical_columns]) 


##############################################################################
##############################################################################
##############################################################################
##############################################################################




In [ ]:
train_dataset_no_nans.isna().sum()

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
dtype: int64

## 4. Encoding categorical Data

Data Preprocessing in machine learning requires values of the data in numerical form. Therefore text values in the columns of datasets must be converted into numerical form. 

###4.1 Converting categorical to numerical values

Given the original dataset, it is clear we have a few categorical features. All these need to be encoded. The [LabelEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html) class is used to transform the categorical or string values to numerical ones (between 0 and n_classes-1).



In [ ]:
dummy = train_dataset_no_nans.copy() #note we are using the dataset with no missing values

# encode categorical data for the 'Gender' column
from sklearn.preprocessing import LabelEncoder
# create an object of the LabelEncoder class
lblEncoder_X = LabelEncoder()   
# apply LblEncoder object to our categorical variables (columns - 'Gender') using the fit_transform method. This returns the column encoded.
dummy['Gender'] = lblEncoder_X.fit_transform(dummy['Gender']) 

print(dummy)



      Loan_ID  Gender Married  ... Loan_Amount_Term Credit_History Property_Area
0    LP001031       1      No  ...            360.0        1.00000         Urban
1    LP001032       1      No  ...            360.0        1.00000         Urban
2    LP001824       1     Yes  ...            480.0        1.00000     Semiurban
3    LP002928       1     Yes  ...            180.0        1.00000     Semiurban
4    LP001814       1     Yes  ...            360.0        1.00000         Urban
..        ...     ...     ...  ...              ...            ...           ...
386  LP001841       1      No  ...            360.0        1.00000         Rural
387  LP002820       1     Yes  ...            360.0        1.00000         Rural
388  LP001744       1      No  ...            360.0        1.00000     Semiurban
389  LP001552       1     Yes  ...            360.0        1.00000     Semiurban
390  LP001553       1     Yes  ...            360.0        0.85641     Semiurban

[391 rows x 12 columns]


Now let us try a different encoding strategy, the one-hot encoding. For each feature, this strategy expands the columns in the dataset to have one column per category. A 0/1 indicator can then be used to mark which category corresponds to each data point.

In [ ]:
from sklearn.preprocessing import OneHotEncoder

oneHotEncoder = OneHotEncoder()                     

# create dataset copy for testing purposes
tmp_train_dataset_no_nans = train_dataset_no_nans.copy()

# create OneHotEncoder object 
# and fit the OneHotEncoder object to feature Gender
onehot_enc = oneHotEncoder.fit(tmp_train_dataset_no_nans[['Gender']])           
print('The categories are: ', onehot_enc.categories_)

# dum is an array of shape (391,2) containing the one-hot encoding of the feature Gender of the dataframe train_dataset_no_nans
# we make a temporary object to be able to manipulate the extra number of columns
dum = onehot_enc.transform(tmp_train_dataset_no_nans[['Gender']]).toarray()           
dum

# we add to the train_dataset_no_nans one feature column called Female and add there the corresponding encoded values
tmp_train_dataset_no_nans['Female'] = dum[:,0]     
# we add to the train_dataset_no_nans one feature column called Male and add there the corresponding encoded values
tmp_train_dataset_no_nans['Male'] = dum[:,1]
# add the 'other' category for gender
tmp_train_dataset_no_nans['other']= dum[:,2]  
     
# we delete the (former) column/feature Gender
tmp_train_dataset_no_nans= tmp_train_dataset_no_nans.drop(columns='Gender',axis=1)  

tmp_train_dataset_no_nans

The categories are:  [array(['Female', 'Male', 'other'], dtype=object)]


,Loan_ID,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Female,Male,other
0,LP001031,No,0,Graduate,No,5134.046512,0.0,125.0,360.0,1.00000,Urban,0.0,1.0,0.0
1,LP001032,No,0,Graduate,No,4950.000000,0.0,125.0,360.0,1.00000,Urban,0.0,1.0,0.0
2,LP001824,Yes,1,Graduate,No,2882.000000,1843.0,123.0,480.0,1.00000,Semiurban,0.0,1.0,0.0
3,LP002928,Yes,0,Graduate,No,3000.000000,3416.0,56.0,180.0,1.00000,Semiurban,0.0,1.0,0.0
4,LP001814,Yes,2,Graduate,No,9703.000000,0.0,112.0,360.0,1.00000,Urban,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
386,LP001841,No,0,Not Graduate,Yes,2583.000000,2167.0,104.0,360.0,1.00000,Rural,0.0,1.0,0.0
387,LP002820,Yes,0,Graduate,No,5923.000000,2054.0,211.0,360.0,1.00000,Rural,0.0,1.0,0.0
388,LP001744,No,0,Graduate,No,2971.000000,2791.0,144.0,360.0,1.00000,Semiurban,0.0,1.0,0.0
389,LP001552,Yes,0,Graduate,No,4583.000000,5625.0,255.0,360.0,1.00000,Semiurban,0.0,1.0,0.0


There is a slightly quicker way of applying the OneHotEncoder. We can use sklearn function `ColumnTransformer`.
This is a function that applies a given transformation to all the columns of a dataset that we specify. It is a pipeline, that is it can incorporate multiple steps, but here we will use it just for the encoding.

In [ ]:
from sklearn.compose import ColumnTransformer
# define the transformation
ct = ColumnTransformer(
    [
        (
            "onehot_categorical", # --> name of the transformation
            OneHotEncoder(), # --> main function to apply
            ["Gender"], #-->columns to apply it to (we can give more than one column at once!)
        ),
    ],
    remainder="passthrough", #--> what to do with the non-transformed columns. passthrough=keep them
    verbose_feature_names_out=False #--> this keeps columns names simple. Try what happens if you set it as True
)

# create dataset copy for testing purposes
tmp_train_dataset_no_nans = train_dataset_no_nans.copy()

# the output is an ARRAY with the encoded columns.
encoded_array= ct.fit_transform(tmp_train_dataset_no_nans) 

# What if we want a dataframe back? We can combine the array with the info about
# the column names stored in ct.get_feature_names_out()
encoded_col_names= ct.get_feature_names_out()
print(encoded_col_names)

pd.DataFrame(encoded_array, columns=encoded_col_names)


['Gender_Female' 'Gender_Male' 'Gender_other' 'Loan_ID' 'Married'
 'Dependents' 'Education' 'Self_Employed' 'ApplicantIncome'
 'CoapplicantIncome' 'LoanAmount' 'Loan_Amount_Term' 'Credit_History'
 'Property_Area']


,Gender_Female,Gender_Male,Gender_other,Loan_ID,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,0,1,0,LP001031,No,0,Graduate,No,5134.05,0,125,360,1,Urban
1,0,1,0,LP001032,No,0,Graduate,No,4950,0,125,360,1,Urban
2,0,1,0,LP001824,Yes,1,Graduate,No,2882,1843,123,480,1,Semiurban
3,0,1,0,LP002928,Yes,0,Graduate,No,3000,3416,56,180,1,Semiurban
4,0,1,0,LP001814,Yes,2,Graduate,No,9703,0,112,360,1,Urban
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
386,0,1,0,LP001841,No,0,Not Graduate,Yes,2583,2167,104,360,1,Rural
387,0,1,0,LP002820,Yes,0,Graduate,No,5923,2054,211,360,1,Rural
388,0,1,0,LP001744,No,0,Graduate,No,2971,2791,144,360,1,Semiurban
389,0,1,0,LP001552,Yes,0,Graduate,No,4583,5625,255,360,1,Semiurban


##############################################################################
##############################################################################
##############################################################################
##############################################################################





##  Exercise no 2

Perform one-hot encoding for ALL the categorical features at once.

Now, work on `train_dataset_no_nans` directly, without creating a copy!

Call the ColumnTransformer object `ct_cat`

In [ ]:
# place your code here

# define the transformation
ct_cat = ColumnTransformer(
    [
        (
            "onehot_categorical", # --> name of the transformation
            OneHotEncoder(), # --> main function to apply
            categorical_columns, #-->columns to apply it to
        ),
    ],
    remainder="passthrough", #--> what to do with the non-transformed columns. passthrough=keep them
    verbose_feature_names_out=False 
)

# the output is an ARRAY with the encoded columns.
encoded_array= ct_cat.fit_transform(train_dataset_no_nans) 

# What if we want a dataframe back? We can combine the array with the info about
# the column names stored in ct.get_feature_names_out()
encoded_col_names= ct_cat.get_feature_names_out()
print(encoded_col_names)

train_dataset_no_nans= pd.DataFrame(encoded_array, columns=encoded_col_names)


##  Exercise no 3

Perform label encoding for the labels (i.e., the train_labels **dataframe**)



In [ ]:
# First check: what are the target categories?
train_labels.value_counts()

In [ ]:
# place your code here for label encoding
# create an object of the LabelEncoder class
lblEncoder_Y = LabelEncoder()   
# apply LblEncoder object to our target variables
train_encoded_labels = lblEncoder_Y.fit_transform(train_labels['Target']) 
train_encoded_labels

##############################################################################
##############################################################################
##############################################################################
##############################################################################





!Note that this procedure does not necessarily maintain the correct datatypes. To be confident in our data, we can reset them as appropriate. For example, we may want to make sure that all the numerical columns are of dtype float.


In [ ]:
# make sure numerical columns are of type float
train_dataset_no_nans= train_dataset_no_nans.astype(dtype={col: "float64" for col in numerical_columns})


##5.  Compute Statistics and Check Imbalance

In [ ]:
# Check whether the problem is unbalance
# That is, check whether some target labels are (much) more common than others
# training dataset
sampleClassBias = train_labels['Target'].value_counts()
print('Training Labels distribution:')
pretty_print_df(sampleClassBias)

# testing dataset
sampleClassBias = test_labels['Target'].value_counts()
print('Test Labels distribution:')
pretty_print_df(sampleClassBias)


Yes - the dataset is unbalanced! There are more labels of type Y than of type N. It's not a massive unbalance, but there are still roughly twice as much "Y" values than there are "N" ones.

We're not going to address it today (lost to go through already), but if you're particularly keen at the end of the tutorial you could have a look at `sklearn.utils.resample` or at one of sklearn contributor package [imbalanced learn](https://github.com/scikit-learn-contrib/imbalanced-learn).

##6.  Feature Scaling

When the data is comprised of feature values with varying scales, many machine learning algorithms can benefit from rescaling the attributes to all have the same scale. 


###6.1 Implementing Feature Scaling

Feature scaling is a method used to scale the range of variables/values of features. 

>![feature scaling](https://drive.google.com/uc?id=1loaMbVo_7ZJo53Ogv7I0XMG_wSOOYBxI)

>  Figure 5: Feature Scaling methods
> Where X is the observation feature.

There are several ways of scaling the data. One way is called **Standardisation** which may be used. For every observation of the selected column, our program will apply the formula of standardisation and fit it to a scale. That is for each observation and each feature within the mean value is withdrawn (subtracted ) from all the values of the feature and divide by the standard deviation. 
The other common type of scaling is **normalisation** where the minimal value of all the feature values is subtracted from the observation feature X and divided by the difference between the max of the feature values and the min of the feature values.

**It is not essential to understand the math behind these methods, what is important is to remember that the variables are being put in the same range / same scale so that no variable is dominated by another.**


In [ ]:
#normalisation - let's work on a copy of the dataset for this one
ddummy = train_dataset_no_nans.copy()


# Importing MinMaxScaler and initializing it
from sklearn.preprocessing import MinMaxScaler
min_max_num=MinMaxScaler()
# Normalising (or MinMax scaling) - let's apply to one column only for now
#NOTE THE DOUBLE SQUARE BRACKETS on the right hand side! 
# Scalers expect a 2-dimensional input: a dataframe is 2-dimensional, a pandas Series is NOT.
# passing one column in double square brackets tells pandas to return a dataframe rather than a series
# try comparing type(ddummy[['ApplicantIncome']]) and type(ddummy['ApplicantIncome'])
ddummy['ApplicantIncome']=min_max_num.fit_transform(ddummy[['ApplicantIncome']]) 

ddummy[['ApplicantIncome']]



Question: Do we need to apply feature scaling to the labels? The answer is no! The label is a categorical value that takes 2 values either no (0) or yes (1).  It is a classification problem.


In [ ]:
# Standardisation - let's work on a copy again
ddummy = train_dataset_no_nans.copy()

#Standardizing (removing mean and dividing by standard deviation)
from sklearn.preprocessing import StandardScaler
scaler_num= StandardScaler()
ddummy['ApplicantIncome']=scaler_num.fit_transform(ddummy[['ApplicantIncome']]) 

ddummy[['ApplicantIncome']]



##############################################################################
##############################################################################
##############################################################################
##############################################################################





##  Exercise no 4

Perform standardization OR normalization on all the numerical features at once.

Work on `train_dataset_no_nans` now!

Call the scaler of your choice `my_scaler_num`


In [ ]:
# place code here
my_scaler_num= StandardScaler()
train_dataset_no_nans[numerical_columns]=my_scaler_num.fit_transform(train_dataset_no_nans[numerical_columns]) 

train_dataset_no_nans[numerical_columns]

##############################################################################
##############################################################################
##############################################################################
##############################################################################





##7. Final

The steps below can be used as a base for ML projects with small variation.

##  Exercise no 5

Perform all the previous data pre-processing steps to the test data and labels.
In order, we did:
1. fill numerical missing values with SimpleImputer (`imptr_num`)
2. fill categorical missing values with SimpleImputer (`imptr_cat`)
3. encoded all categorical values with ColumnTransformer (`ct_cat`)
4. scaled all numerical features with MinMax or StandardScaler (`my_scaler_num`)

Also, we encoded the labels with LabelEncoder (`lblEncoder_Y`)

When transforming the test dataset (both input and labels) we should use transformer objects that have been **fit on the training data**. This is because in real-world applications the test data would not be available during training. 

So, we can call the `.transform` function alone on previously trained object.

In [ ]:
# place code here
test_dataset_no_nans= test_dataset.copy()

# filling nans
test_dataset_no_nans[numerical_columns] = imptr_num.transform(test_dataset_no_nans[numerical_columns])
test_dataset_no_nans[categorical_columns] = imptr_cat.transform(test_dataset_no_nans[categorical_columns])

# encoding categorical values
encoded_test_array= ct_cat.transform(test_dataset_no_nans) 

test_dataset_no_nans= pd.DataFrame(encoded_test_array, columns=ct_cat.get_feature_names_out())

# make sure numerical columns are of type float
test_dataset_no_nans= test_dataset_no_nans.astype(dtype={col: "float64" for col in numerical_columns})

# scaling
test_dataset_no_nans[numerical_columns] = my_scaler_num.transform(test_dataset_no_nans[numerical_columns])

# encode labels
test_encoded_labels = lblEncoder_Y.transform(test_labels['Target']) 


Next I provide a standard classifier so that you perform some tests

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score


# we will use a classifier (we will skip details and what it does for now, so just use it as it is)
knn=KNeighborsClassifier(n_neighbors=5)
knn.fit(train_dataset_no_nans.drop(columns=['Loan_ID']), train_encoded_labels) #train_labels['Target'])  
## we train the classifier with the training data and labels: train_dataset_no_nans1 should be the training dataframe after:  
## i) filling in all missing values, ii) encoding all categorical features and (maybe) after feature scaling

# Checking the model's accuracy (performance); this should be performed on the test set and thus we use the test_dataset_no_nans1 (same pre-processing as before shoud have been performed) and the test labels (after encoding)
#accuracy_score(test_labels['Target'], knn.predict(test_dataset_no_nans.drop(columns=['Loan_ID'])))
accuracy_score(test_encoded_labels, knn.predict(test_dataset_no_nans.drop(columns=['Loan_ID'])))


## [Stretch goal] Final Exercise 

Test the performance of the classifier when:

- the label encoder has been used and no feature scaling has been performed
- the label encoder has been used and normalisation has been performed 
- the label encoder has been used and standardisation has been performed 
- the one-hot encoder has been used and no feature scaling has been performed
- the one-hot encoder has been used and normalisation has been performed 
- the one-hot encoder has been used and standardisation has been performed 

Maybe the performance can be further improved? Have a look at the features that you used and think what data quality assessment procedures can be performed.